# Understanding Classes and Object-Oriented Programming

This notebook teaches you **why classes are useful** and **how to use them effectively** in the context of the Simple Autonomous Car SDK.

## What You'll Learn

1. **Why Classes?** - Understanding the benefits of OOP
2. **Class Basics** - Attributes, methods, and instances
3. **Inheritance** - Building on existing classes
4. **Abstract Base Classes** - Defining interfaces
5. **Polymorphism** - One interface, many implementations
6. **Real Examples** - How the SDK uses classes

## Prerequisites

- Basic Python knowledge (variables, functions, lists)
- Understanding of dictionaries and data structures
- No prior OOP experience required!

## 1. Why Classes? The Problem They Solve

Let's start with a **real problem** from autonomous driving:

### Problem: Managing Multiple Controllers

Imagine you want to test different control algorithms:
- Pure Pursuit controller
- PID controller  
- Model Predictive Control (MPC)

**Without classes**, you might write code like this:

In [ ]:
# Without classes - messy approach
def pure_pursuit_control(car_state, plan, lookahead=10.0, target_velocity=10.0):
    # Pure pursuit logic here
    return {"acceleration": 0.5, "steering_rate": 0.1}

def pid_control(car_state, plan, kp=1.0, ki=0.1, kd=0.01):
    # PID logic here
    return {"acceleration": 0.5, "steering_rate": 0.1}

def mpc_control(car_state, plan, horizon=10, dt=0.1):
    # MPC logic here
    return {"acceleration": 0.5, "steering_rate": 0.1}

# Problems:
# 1. Each function has different parameters - hard to swap them
# 2. No way to store state (e.g., PID integral term)
# 3. Can't easily extend or modify behavior
# 4. Hard to organize related functionality

### Solution: Classes

**With classes**, we can:
1. **Group related data and functions** together
2. **Store state** between function calls
3. **Create multiple instances** with different configurations
4. **Extend functionality** through inheritance

Let's see how the SDK does it:

In [ ]:
import sys
sys.path.insert(0, '../../../src')

import numpy as np
from simple_autonomous_car import (
    Car,
    CarState,
    PurePursuitController,
    PIDController,
    Track,
    TrackPlanner,
)

# Create different controllers with different configurations
controller1 = PurePursuitController(
    lookahead_distance=10.0,
    target_velocity=15.0,
    name="aggressive_pursuit"
)

controller2 = PurePursuitController(
    lookahead_distance=20.0,
    target_velocity=8.0,
    name="smooth_pursuit"
)

controller3 = PIDController(
    kp=1.5,
    ki=0.2,
    kd=0.05,
    name="pid_controller"
)

print(f"Controller 1: {controller1.name}, lookahead={controller1.lookahead_distance}")
print(f"Controller 2: {controller2.name}, lookahead={controller2.lookahead_distance}")
print(f"Controller 3: {controller3.name}")

# All controllers have the same interface - easy to swap!
car = Car(initial_state=CarState(x=0, y=0, heading=0, velocity=10.0))
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)
planner = TrackPlanner(track)
plan = planner.plan(car.state)

# Use any controller with the same code
control1 = controller1.compute_control(car.state, plan=plan)
control2 = controller2.compute_control(car.state, plan=plan)
control3 = controller3.compute_control(car.state, plan=plan)

print(f"\nControl 1: {control1}")
print(f"Control 2: {control2}")
print(f"Control 3: {control3}")

## 2. Class Basics: Attributes and Methods

A **class** is like a blueprint. An **instance** (object) is a specific thing built from that blueprint.

### Example: CarState Class

Let's examine a simple class from the SDK:

In [ ]:
from simple_autonomous_car import CarState

# Create an instance (object) of CarState
state1 = CarState(x=10.0, y=20.0, heading=0.5, velocity=15.0)

# Access attributes (data stored in the object)
print(f"Position: ({state1.x}, {state1.y})")
print(f"Heading: {state1.heading} radians")
print(f"Velocity: {state1.velocity} m/s")

# Call methods (functions that belong to the object)
position = state1.position()  # Returns [x, y] array
print(f"\nPosition array: {position}")

# Create another instance with different values
state2 = CarState(x=5.0, y=10.0, heading=1.0, velocity=8.0)
print(f"\nState 2 position: {state2.position()}")

# Each instance is independent
print(f"\nState 1 and State 2 are different: {state1.x != state2.x}")

### Understanding `self`

Inside a class, `self` refers to the **current instance**. Let's see how methods work:

In [ ]:
# Let's look at what methods CarState has
print("CarState methods:")
for method_name in dir(state1):
    if not method_name.startswith('_'):  # Skip private methods
        print(f"  - {method_name}")

# Try some methods
print(f"\nPosition: {state1.position()}")
print(f"Transform to car frame [0, 0]: {state1.transform_to_car_frame([0, 0])}")

# Methods can modify or use the object's data
transformed_point = state1.transform_to_car_frame([10, 5])
print(f"\nPoint [10, 5] in car frame: {transformed_point}")

## 3. Inheritance: Building on Existing Classes

**Inheritance** lets you create a new class based on an existing one. The new class:
- **Inherits** all attributes and methods from the parent
- Can **add new** attributes and methods
- Can **override** (replace) parent methods

### Example: BaseController → PurePursuitController

Let's see the inheritance hierarchy:

In [ ]:
from simple_autonomous_car.control.base_controller import BaseController
from simple_autonomous_car.control.pure_pursuit_controller import PurePursuitController

# Check inheritance
print(f"PurePursuitController is a BaseController: {issubclass(PurePursuitController, BaseController)}")
print(f"PurePursuitController inherits from: {PurePursuitController.__bases__}")

# Create an instance
controller = PurePursuitController()

# Methods inherited from BaseController
print(f"\nController name: {controller.name}")
print(f"Controller enabled: {controller.enabled}")
print(f"\nIs enabled? {controller.is_enabled()}")

# Methods specific to PurePursuitController
print(f"Lookahead distance: {controller.lookahead_distance}")
print(f"Target velocity: {controller.target_velocity}")

# Method overridden in PurePursuitController
car = Car(initial_state=CarState(x=0, y=0, heading=0, velocity=10.0))
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)
planner = TrackPlanner(track)
plan = planner.plan(car.state)

control = controller.compute_control(car.state, plan=plan)
print(f"\nControl output: {control}")

### Why Inheritance is Powerful

**All controllers** share common functionality from `BaseController`:
- `name` and `enabled` attributes
- `enable()`, `disable()`, `is_enabled()` methods
- `compute_control()` interface (must be implemented)
- `visualize()` method

This means you can **swap controllers** without changing your code!

In [ ]:
def test_controller(controller, car_state, plan):
    """Test any controller - works because they all inherit from BaseController"""
    if not controller.is_enabled():
        print(f"{controller.name} is disabled!")
        return None
    
    control = controller.compute_control(car_state, plan=plan)
    print(f"{controller.name} computed: {control}")
    return control

# Test with different controllers
car_state = CarState(x=0, y=0, heading=0, velocity=10.0)
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)
planner = TrackPlanner(track)
plan = planner.plan(car_state)

controllers = [
    PurePursuitController(name="pursuit_1"),
    PurePursuitController(lookahead_distance=20.0, name="pursuit_2"),
    PIDController(name="pid_1"),
]

print("Testing multiple controllers with the same function:\n")
for controller in controllers:
    test_controller(controller, car_state, plan)
    print()

## 4. Abstract Base Classes: Defining Interfaces

An **Abstract Base Class (ABC)** defines what methods a class **must** implement, but doesn't provide the implementation itself.

### Why Use ABCs?

1. **Guarantee consistency** - All subclasses have the same interface
2. **Documentation** - Clear contract of what methods are needed
3. **Error prevention** - Python will error if you forget to implement required methods

### Example: BaseController

In [ ]:
from abc import ABC, abstractmethod
from simple_autonomous_car.control.base_controller import BaseController

# BaseController is an abstract class
print(f"BaseController is abstract: {ABC in BaseController.__bases__}")

# Try to create BaseController directly - this will fail!
try:
    base = BaseController()
    base.compute_control(car.state)  # This would fail - method not implemented
except Exception as e:
    print(f"\nCannot use BaseController directly: {type(e).__name__}")

# But we can use concrete implementations
print("\nConcrete controllers work:")
pursuit = PurePursuitController()
pid = PIDController()

print(f"PurePursuitController has compute_control: {hasattr(pursuit, 'compute_control')}")
print(f"PIDController has compute_control: {hasattr(pid, 'compute_control')}")

# Both implement the required interface
plan = planner.plan(car.state)
print(f"\nPursuit control: {pursuit.compute_control(car.state, plan=plan)}")
print(f"PID control: {pid.compute_control(car.state, plan=plan)}")

### Creating Your Own Controller

Let's create a simple controller by inheriting from `BaseController`:

In [ ]:
from simple_autonomous_car.control.base_controller import BaseController
from simple_autonomous_car.car.car import CarState
import numpy as np

class SimpleController(BaseController):
    """A very simple controller that just goes straight."""
    
    def __init__(self, target_velocity=10.0, name="simple"):
        super().__init__(name=name)  # Call parent __init__
        self.target_velocity = target_velocity
    
    def compute_control(self, car_state, perception_data=None, 
                       costmap=None, plan=None, dt=0.1):
        """Must implement this method (required by BaseController)."""
        # Simple: maintain target velocity, no steering
        current_velocity = car_state.velocity
        velocity_error = self.target_velocity - current_velocity
        
        # Simple proportional control
        acceleration = 0.5 * velocity_error
        
        return {
            "acceleration": acceleration,
            "steering_rate": 0.0  # Go straight
        }

# Now we can use it just like any other controller!
simple = SimpleController(target_velocity=12.0, name="my_simple")
print(f"Controller name: {simple.name}")
print(f"Target velocity: {simple.target_velocity}")

# It works with the same interface
car_state = CarState(x=0, y=0, heading=0, velocity=8.0)
control = simple.compute_control(car_state)
print(f"\nControl output: {control}")

# Can use it in our test function too!
plan = planner.plan(car_state)
test_controller(simple, car_state, plan)

## 5. Polymorphism: One Interface, Many Implementations

**Polymorphism** means "many forms". In OOP, it means you can use different classes through the same interface.

### Example: Different Planners, Same Interface

In [ ]:
from simple_autonomous_car.planning.base_planner import BasePlanner
from simple_autonomous_car.planning.track_planner import TrackPlanner

def use_planner(planner: BasePlanner, car_state: CarState):
    """This function works with ANY planner that inherits from BasePlanner."""
    print(f"Using planner: {planner.name}")
    print(f"Planner enabled: {planner.is_enabled()}")
    
    # All planners have the same interface
    plan = planner.plan(car_state)
    print(f"Generated {len(plan)} waypoints")
    return plan

# Create different planners
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)
planner1 = TrackPlanner(track, lookahead_distance=30.0, name="short_range")
planner2 = TrackPlanner(track, lookahead_distance=70.0, name="long_range")

car_state = CarState(x=0, y=0, heading=0, velocity=10.0)

# Same function works with different planners!
print("=== Planner 1 ===")
plan1 = use_planner(planner1, car_state)

print("\n=== Planner 2 ===")
plan2 = use_planner(planner2, car_state)

print(f"\nPlanner 1 generated {len(plan1)} waypoints")
print(f"Planner 2 generated {len(plan2)} waypoints")

### Real-World Example: Simulation Loop

Here's how polymorphism makes the simulation code clean and flexible:

In [ ]:
def simulation_step(car, planner, controller, track):
    """One simulation step - works with ANY planner and controller!"""
    # 1. Generate plan (works with any BasePlanner)
    plan = planner.plan(car.state)
    
    # 2. Compute control (works with any BaseController)
    control = controller.compute_control(car.state, plan=plan)
    
    # 3. Update car state
    car.state.velocity += control["acceleration"] * 0.1
    car.state.steering_angle += control["steering_rate"] * 0.1
    
    return plan, control

# Try different combinations
car = Car(initial_state=CarState(x=0, y=0, heading=0, velocity=10.0))
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)

# Combination 1: TrackPlanner + PurePursuitController
planner1 = TrackPlanner(track, name="track_planner")
controller1 = PurePursuitController(name="pursuit")
plan1, control1 = simulation_step(car, planner1, controller1, track)
print(f"Combo 1: {planner1.name} + {controller1.name}")
print(f"  Plan: {len(plan1)} waypoints, Control: {control1}")

# Combination 2: TrackPlanner + PIDController
controller2 = PIDController(name="pid")
plan2, control2 = simulation_step(car, planner2, controller2, track)
print(f"\nCombo 2: {planner2.name} + {controller2.name}")
print(f"  Plan: {len(plan2)} waypoints, Control: {control2}")

# The beauty: We can swap components without changing simulation_step!

## 6. Class Hierarchy in the SDK

Let's explore the complete class structure:

In [ ]:
import inspect

def print_class_hierarchy(cls, indent=0):
    """Print the inheritance hierarchy of a class."""
    print("  " * indent + cls.__name__)
    for base in cls.__bases__:
        if base != object:
            print_class_hierarchy(base, indent + 1)

print("=== Controller Hierarchy ===")
print_class_hierarchy(PurePursuitController)

print("\n=== Planner Hierarchy ===")
print_class_hierarchy(TrackPlanner)

print("\n=== Sensor Hierarchy ===")
from simple_autonomous_car.sensors.lidar_sensor import LiDARSensor
print_class_hierarchy(LiDARSensor)

print("\n=== Costmap Hierarchy ===")
from simple_autonomous_car.costmap.grid_costmap import GridCostmap
print_class_hierarchy(GridCostmap)

## 7. Practical Exercise: Build a Custom Component

Let's build a simple controller step by step:

In [ ]:
# Step 1: Import the base class
from simple_autonomous_car.control.base_controller import BaseController
import numpy as np

# Step 2: Create your class inheriting from BaseController
class MyFirstController(BaseController):
    """A controller that follows the plan by pointing towards the next waypoint."""
    
    def __init__(self, lookahead_idx=5, name="my_first"):
        # Always call super().__init__() first
        super().__init__(name=name)
        
        # Add your own attributes
        self.lookahead_idx = lookahead_idx  # Which waypoint to target
    
    def compute_control(self, car_state, perception_data=None, 
                       costmap=None, plan=None, dt=0.1):
        """Implement the required method."""
        
        # Check if we have a plan
        if plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}
        
        # Get target waypoint
        target_idx = min(self.lookahead_idx, len(plan) - 1)
        target = plan[target_idx]
        
        # Calculate direction to target
        car_pos = car_state.position()
        direction = target - car_pos
        distance = np.linalg.norm(direction)
        
        if distance < 0.1:
            return {"acceleration": 0.0, "steering_rate": 0.0}
        
        # Calculate desired heading
        desired_heading = np.arctan2(direction[1], direction[0])
        
        # Calculate heading error
        heading_error = desired_heading - car_state.heading
        # Normalize to [-pi, pi]
        heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))
        
        # Simple proportional control
        steering_rate = 2.0 * heading_error  # Gain of 2.0
        
        # Maintain constant velocity
        target_velocity = 10.0
        velocity_error = target_velocity - car_state.velocity
        acceleration = 0.5 * velocity_error
        
        return {
            "acceleration": acceleration,
            "steering_rate": steering_rate
        }

# Step 3: Test it!
my_controller = MyFirstController(lookahead_idx=3, name="my_controller")

car = Car(initial_state=CarState(x=0, y=0, heading=0, velocity=10.0))
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)
planner = TrackPlanner(track)
plan = planner.plan(car.state)

control = my_controller.compute_control(car.state, plan=plan)
print(f"My controller computed: {control}")

# It works with our test function too!
test_controller(my_controller, car.state, plan)

## 8. Key Takeaways

### Why Classes?

1. **Organization**: Group related data and functions together
2. **Reusability**: Create multiple instances with different configurations
3. **State Management**: Store data between function calls
4. **Extensibility**: Build on existing classes through inheritance
5. **Polymorphism**: Use different implementations through the same interface

### Best Practices

1. **Inherit from base classes** - Use `BaseController`, `BasePlanner`, etc.
2. **Implement required methods** - Abstract methods must be implemented
3. **Call `super().__init__()`** - Initialize parent class first
4. **Use consistent interfaces** - Follow the same method signatures
5. **Document your classes** - Add docstrings explaining what they do

### Next Steps

- Try building your own sensor: [Building a Custom Sensor](../sensors/learning_build_sensor.ipynb)
- Try building your own controller: [Building a Custom Controller](../control/learning_build_controller.ipynb)
- Try building your own planner: [Building a Custom Planner](../planning/learning_build_planner.ipynb)
- Explore advanced algorithms: [Advanced Planning Algorithms](../algorithms/advanced_planning_algorithms.ipynb)

## 9. Summary

Classes and OOP provide a powerful way to:

- **Organize code** into logical units
- **Create reusable components** that can be configured differently
- **Build complex systems** from simple building blocks
- **Swap implementations** without changing your code

The Simple Autonomous Car SDK uses these principles to make it easy to:
- Test different algorithms
- Combine components in new ways
- Extend functionality
- Build complex autonomous systems

**Happy coding!** 🚗